In [1]:

import numpy as np
import time as tm
import datetime as dt
import tensorflow as tf

# Data preparation

from sklearn.preprocessing import MinMaxScaler
from collections import deque

# AI
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

# Graphics library
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# SETTINGS

# Window size or the sequence length, 7 (1 week)
N_STEPS = 7

# Lookup steps, 1 is the next day, 3 = after tomorrow
LOOKUP_STEPS = [i for i in range(1, 51)]
# LOOKUP_STEPS = [1,2,3]

df = pd.read_csv('NFLX.csv')
df.set_index('Date', inplace=True)
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-02-05,262.000000,267.899994,250.029999,254.259995,254.259995,11896100
2018-02-06,247.699997,266.700012,245.000000,265.720001,265.720001,12595800
2018-02-07,266.579987,272.450012,264.329987,264.559998,264.559998,8981500
2018-02-08,267.079987,267.619995,250.000000,250.100006,250.100006,9306700
2018-02-09,253.850006,255.800003,236.110001,249.470001,249.470001,16906900
...,...,...,...,...,...,...
2022-01-31,401.970001,427.700012,398.200012,427.140015,427.140015,20047500
2022-02-01,432.959991,458.480011,425.540009,457.130005,457.130005,22542300
2022-02-02,448.250000,451.980011,426.480011,429.480011,429.480011,14346000


In [2]:
init_df = df.drop(['Open', 'High', 'Low','Adj Close','Volume'], axis=1)
init_df['date'] = init_df.index

init_df

,Close,date
Date,,
2018-02-05,254.259995,2018-02-05
2018-02-06,265.720001,2018-02-06
2018-02-07,264.559998,2018-02-07
2018-02-08,250.100006,2018-02-08
2018-02-09,249.470001,2018-02-09
...,...,...
2022-01-31,427.140015,2022-01-31
2022-02-01,457.130005,2022-02-01
2022-02-02,429.480011,2022-02-02


In [3]:
scaler = MinMaxScaler()
init_df['Close'] = scaler.fit_transform(np.expand_dims(init_df['Close'].values, axis=1))

init_df

,Close,date
Date,,
2018-02-05,0.044516,2018-02-05
2018-02-06,0.069548,2018-02-06
2018-02-07,0.067015,2018-02-07
2018-02-08,0.035430,2018-02-08
2018-02-09,0.034053,2018-02-09
...,...,...
2022-01-31,0.422140,2022-01-31
2022-02-01,0.487648,2022-02-01
2022-02-02,0.427251,2022-02-02


In [4]:
def PrepareData(days):
  df = init_df.copy()
  df['future'] = df['Close'].shift(-days)
  last_sequence = np.array(df[['Close']].tail(days))
  df.dropna(inplace=True)
  sequence_data = []
  sequences = deque(maxlen=N_STEPS)

  for entry, target in zip(df[['Close'] + ['date']].values, df['future'].values):
      sequences.append(entry)
      if len(sequences) == N_STEPS:
          sequence_data.append([np.array(sequences), target])

  last_sequence = list([s[:len(['Close'])] for s in sequences]) + list(last_sequence)
  last_sequence = np.array(last_sequence).astype(np.float32)

  # construct the X's and Y's
  X, Y = [], []
  for seq, target in sequence_data:
      X.append(seq)
      Y.append(target)

  # convert to numpy arrays
  X = np.array(X)
  Y = np.array(Y)

  return df, last_sequence, X, Y

In [5]:
def GetTrainedModel(x_train, y_train):
  model = Sequential()
  model.add(LSTM(60, return_sequences=True, input_shape=(N_STEPS, len(['Close']))))
  model.add(Dropout(0.3))
  model.add(LSTM(120, return_sequences=False))
  model.add(Dropout(0.3))
  model.add(Dense(20))
  model.add(Dense(1))

  BATCH_SIZE = 8
  EPOCHS = 20

  model.compile(loss='mean_squared_error', optimizer='adam')

  model.fit(x_train, y_train,
            batch_size=BATCH_SIZE,
            epochs=EPOCHS,
            verbose=1)

  model.summary()

  return model

In [6]:
predictions = []

for step in LOOKUP_STEPS:
  df, last_sequence, x_train, y_train = PrepareData(step)
  x_train = x_train[:, :, :len(['Close'])].astype(np.float32)

  model = GetTrainedModel(x_train, y_train)

  last_sequence = last_sequence[-N_STEPS:]
  last_sequence = np.expand_dims(last_sequence, axis=0)
  prediction = model.predict(last_sequence)
  predicted_price = scaler.inverse_transform(prediction)[0][0]

  predictions.append(round(float(predicted_price), 2))

Epoch 1/20
126/126 [==============================] - 3s 5ms/step - loss: 0.0094
Epoch 2/20
126/126 [==============================] - 1s 4ms/step - loss: 0.0049
Epoch 3/20
126/126 [==============================] - 1s 4ms/step - loss: 0.0030
Epoch 4/20
126/126 [==============================] - 1s 4ms/step - loss: 0.0033
Epoch 5/20
126/126 [==============================] - 1s 4ms/step - loss: 0.0029
Epoch 6/20
126/126 [==============================] - 1s 4ms/step - loss: 0.0027
Epoch 7/20
126/126 [==============================] - 1s 4ms/step - loss: 0.0022
Epoch 8/20
126/126 [==============================] - 1s 4ms/step - loss: 0.0024
Epoch 9/20
126/126 [==============================] - 1s 4ms/step - loss: 0.0020
Epoch 10/20
126/126 [==============================] - 1s 4ms/step - loss: 0.0022
Epoch 11/20
126/126 [==============================] - 1s 4ms/step - loss: 0.0018
Epoch 12/20
126/126 [==============================] - 1s 4ms/step - loss: 0.0019
Epoch 13/20
126/126 [====

KeyboardInterrupt: 

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

In [ ]:
# y_predicted = model.predict(x_train)
# y_predicted_transformed = np.squeeze(scaler.inverse_transform(y_predicted))

# plt.figure(figsize=(16,10))
# plt.plot(df['Close'].tolist())
# plt.plot(y_predicted_transformed)



32/32 [==============================] - 0s 2ms/step


ValueError: Must have equal len keys and value when setting with an iterable